In [1]:
import tensorflow as tf
import numpy as np
import timeit
from sklearn.utils import shuffle
#import matplotlib.pyplot as plt
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h1>Extract MNIST data</h1>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding, reshape = False (that means images are not flatten)
mnist = input_data.read_data_sets("MNIST_data/",reshape=False,one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h1>Prepare training, validation and testing data</h1>

In [3]:
x_train, y_train           = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test             = mnist.test.images, mnist.test.labels

#pad images with 0s (28x28 to 32x32)
x_train_pad = np.pad(x_train, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)
x_validation_pad = np.pad(x_validation, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)
x_test_pad = np.pad(x_test, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)

# will use a generator function...
def get_batches(datax, datay, batch_size):
    idx = 0
    # need to shuffle here, but its chill
    while idx * batch_size < datax.shape[0]:
        start = idx * batch_size
        end = idx * batch_size + batch_size
        yield (datax[start:end], datay[start:end])
        idx += 1


<h1>Define hyperparameter</h1>

In [21]:
batch_size, num_epochs = 128, 30

training_batches = get_batches(x_train_pad, y_train, batch_size)

In [22]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [23]:
x_in = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))
y_true = tf.placeholder(tf.float32, shape=(None, 10))

<h1>Define LeNet-5</h1>

In [24]:
# LeNet 5 implements trainable average pooling layers
# in addition to convolutional layers that dont convolve over the
# whole feature map. Here we omit these features and
# copy the general structure, using max pool, and full conv layers
# They also use sigmoids in the convolution, omitted here
# and RBF for loss. we just use cross entropy

# c1: convolution2d
c1 = tf.layers.conv2d(x_in, filters=6, kernel_size=5, activation=tf.nn.sigmoid)

# s1: max pool substituted for trainable average pool
s1 = tf.layers.max_pooling2d(c1, pool_size=2, strides=2)

# c2: convolution 2d size (5,5, 10)
c2 = tf.layers.conv2d(s1, filters=16, kernel_size=5, activation=tf.nn.sigmoid)

# s3: max pool substitute for ave pool
s2 = tf.layers.max_pooling2d(c2, pool_size=2, strides=2)

# c4: 1x1 conv layer
c3 = tf.layers.conv2d(s2, filters=120, kernel_size=1, activation=tf.nn.sigmoid)

# flatten for fully connect
c3_flat = tf.layers.flatten(c3)

# dense
d4 = tf.layers.dense(c3_flat, units=84, activation=tf.nn.sigmoid)

# output
out = tf.layers.dense(d4, units=10)


<h1>Cost and optimization</h1>

In [25]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=out))
opt = tf.train.AdamOptimizer()
to_minimize = opt.minimize(cost)

# compare prediction accuracy 
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(out),1),tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')


<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [26]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        print("Training on Epoch \t" + str(i))
        training_batches = get_batches(x_train_pad, y_train, batch_size)
        for x_data, y_labels in training_batches:
            f_dict = {x_in: x_data, y_true: y_labels}
            sess.run(to_minimize, feed_dict=f_dict)
        current_acc = sess.run(accuracy, feed_dict=f_dict)
        print(current_acc)
        val_acc = sess.run(accuracy, feed_dict={x_in:x_validation_pad, y_true:y_validation})
        print('Val Acc')
        print(val_acc)
    test_acc = sess.run(accuracy, feed_dict={x_in:x_test_pad, y_true:y_test})
    print(test_acc)

Training on Epoch 	0
0.10227273
Val Acc
0.1126
Training on Epoch 	1
0.10227273
Val Acc
0.1126
Training on Epoch 	2
0.79545456
Val Acc
0.7476
Training on Epoch 	3
0.9431818
Val Acc
0.915
Training on Epoch 	4
0.97727275
Val Acc
0.9386
Training on Epoch 	5
0.97727275
Val Acc
0.9522
Training on Epoch 	6
0.97727275
Val Acc
0.9604
Training on Epoch 	7
0.96590906
Val Acc
0.9652
Training on Epoch 	8
0.97727275
Val Acc
0.969
Training on Epoch 	9
0.97727275
Val Acc
0.973
Training on Epoch 	10
0.97727275
Val Acc
0.9752
Training on Epoch 	11
0.97727275
Val Acc
0.977
Training on Epoch 	12
0.9886364
Val Acc
0.979
Training on Epoch 	13
0.9886364
Val Acc
0.9804
Training on Epoch 	14
0.9886364
Val Acc
0.9806
Training on Epoch 	15
0.9886364
Val Acc
0.9808
Training on Epoch 	16
0.9886364
Val Acc
0.9822
Training on Epoch 	17
0.9886364
Val Acc
0.9822
Training on Epoch 	18
0.9886364
Val Acc
0.9828
Training on Epoch 	19
0.9886364
Val Acc
0.9826
Training on Epoch 	20
0.9886364
Val Acc
0.9824
Training on Epoch